In [1]:
using PyPlot, Distributions, Optim, TimeIt
include("ZZ_subsampling.jl") 

acf (generic function with 1 method)

In [2]:
function logistic_data(d, Nobs, p=0.1) 
    X = ones(d, Nobs)
    X[2:(end-1),:] = rand(d-2, Nobs)
    X[:,end] = rand(Normal(),d)
    y = rand(Binomial(1,p), Nobs)
    return X, y
end

logistic_data (generic function with 2 methods)

In [3]:
d, Nobs = 10, 100
X, y = logistic_data(d, Nobs, 0.3)
X .*= rand(Binomial(1,0.4), d, Nobs)
sum(y)

32

In [4]:
σ2 = rand(Uniform(2,5), d)
A = eye(d)
root = find_root(X, y, σ2);

In [5]:
ϵ = 1e-5
weights_cv = zeros(d, Nobs)
for n in 1:Nobs
    weights_cv[:,n] = [abs.(X[i,n])*norm(X[:,n]) for i in 1:d] + ϵ
end
weights_cv ./= sum(weights_cv,2);
weights = abs.(X) + ϵ
weights ./= sum(weights,2);

In [6]:
h = 1
replace = false 

false

In [11]:
mb_size = 10
max_time = 5e10
max_attempts = 10^5
β_0 = zeros(d)
θ = ones(d);

#### Without control variates:

In [12]:
start = time()
s_iid, b_iid, ξ, θ, _1, _2 = 
ZZ_logistic(X, y, max_time, max_attempts, β_0, θ, mb_size, root, σ2, A, false, nothing, replace)

s_unweighted, b_unweighted, ξ, θ, _1, _2 = 
ZZ_logistic(X, y, max_time, max_attempts, β_0, θ, mb_size, root, σ2, A, false, ones(d,Nobs), replace)

s_weighted, b_weighted, ξ, θ, _1, _2 = 
ZZ_logistic(X, y, max_time, max_attempts, β_0, θ, mb_size, root, σ2, A, false, weights, replace)

print( round((time()-start)/60, 2), " minutes to run \n")
samples_iid = extract_samples(s_iid, b_iid, h)
samples_unweighted = extract_samples(s_unweighted, b_unweighted, h)
samples_weighted = extract_samples(s_weighted, b_weighted, h)
print("Configuration temperatures: \n")
print("iid       : ", round.([compute_configT(samples_iid, i, X, y, Nobs, σ2) for i in 1:d], 2), "\n")
print("weighted  : ", round.([compute_configT(samples_weighted, i, X, y, Nobs, σ2) for i in 1:d], 2), "\n")
print("unweighted: ", round.([compute_configT(samples_unweighted, i, X, y, Nobs, σ2) for i in 1:d], 2)) 

0.14 minutes to run 
Configuration temperatures: 
iid       : [0.88, 0.96, 0.85, 0.75, 0.75, 1.15, 0.86, 1.17, 1.08, 1.35]
weighted  : [1.03, 1.05, 1.05, 1.07, 0.99, 0.89, 0.94, 1.02, 0.95, 0.81]
unweighted: [0.97, 1.0, 0.64, 0.72, 0.83, 0.99, 1.2, 0.99, 0.68, 1.52]

In [16]:
b_iid[end], b_unweighted[end], b_weighted[end]

(99.52952572130069, 99.67145625475544, 424.5267341741423)

#### With control variates:

In [13]:
start = time()
s_cv_unweighted, b_cv_unweighted, ξ, θ, _1, _2 = 
ZZ_logistic(X, y, max_time, max_attempts, β_0, θ, mb_size, root, σ2, A, true, nothing, replace)

s_cv_weighted, b_cv_weighted, ξ, θ, _1, _2 = 
ZZ_logistic(X, y, max_time, max_attempts, β_0, θ, mb_size, root, σ2, A, true, weights_cv, replace)

print( round((time()-start)/60, 2), " minutes to run \n")
samples_cv_unweighted = extract_samples(s_cv_unweighted, b_cv_unweighted, h)
samples_cv_weighted = extract_samples(s_cv_weighted, b_cv_weighted, h);
print("Configuration temperatures: \n")
print("CV unweighted: ", round.([compute_configT(samples_cv_unweighted, i, X, y, Nobs, σ2) for i in 1:d],2), "\n")
print("CV weighted  : ", round.([compute_configT(samples_cv_weighted, i, X, y, Nobs, σ2) for i in 1:d],2))

0.03 minutes to run 
Configuration temperatures: 
CV unweighted: [0.95, 1.12, 1.14, 0.8, 1.15, 1.05, 1.3, 0.92, 0.98, 1.07]
CV weighted  : [0.92, 0.97, 1.03, 1.01, 0.99, 0.95, 0.99, 0.88, 0.98, 0.83]

In [14]:
b_cv_unweighted[end], b_cv_weighted[end]

(98.75417957435022, 542.8510933146902)